In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import string
from time import sleep
import sys
import MeCab
import numpy as np
from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
import warnings
%matplotlib inline
warnings.simplefilter('ignore')

# 読み込み

In [2]:
df = pd.read_csv('lyrics_original.csv')
df.head()

,Unnamed: 0,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number
0,0,B'z,Takahiro Matsumoto,Kohshi Inaba,I WANNA DANCE Wicked Beat Style,https://www.uta-net.com/song/54892/,Crush! Crush! Taxi on a busy neon streetGive m...,：1990-06-2,：BMCR-900
1,1,B'z,松本孝弘,稲葉浩志,あいかわらずなボクら,https://www.uta-net.com/song/7938/,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...,：1991-11-2,：BMCR-64B
2,2,B'z,TAK MATSUMOTO,KOSHI INABA,愛と憎しみのハジマリ,https://www.uta-net.com/song/17979/,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...,：2003-09-1,：BMCV-800
3,3,B'z,松本孝弘,稲葉浩志,愛のバクダン,https://www.uta-net.com/song/25415/,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...,：2005-03-0,：BMCV-400
4,4,B'z,TAK MATSUMOTO,KOSHI INABA,愛のprisoner,https://www.uta-net.com/song/26049/,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど...",：2000-12-0,：BMCR-704


# 形態素解析

In [39]:
def get_dokuritsugo_by_mecab(text):
    tagger = MeCab.Tagger('/usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    tagger.parse('')
    node = tagger.parseToNode(text)
    word_list = []
    while node:
        pos = node.feature.split(",")[0]
        if pos in ['名詞', '動詞', '形容詞']:  # 対象とする品詞
            word = node.surface
            word_list.append(word)
        node = node.next
    return ' '.join(word_list)

In [56]:
df['preprocessed_by_mecab'] = df['Lyric'].apply(lambda x:get_dokuritsugo_by_mecab(x))
df.head()

,Unnamed: 0,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number,preprocessed_by_mecab,remove_stop_words
0,0,B'z,Takahiro Matsumoto,Kohshi Inaba,I WANNA DANCE Wicked Beat Style,https://www.uta-net.com/song/54892/,Crush! Crush! Taxi on a busy neon streetGive m...,：1990-06-2,：BMCR-900,Crush ! Crush ! Taxi on a busy neon streetGive...,crush crush taxi busy neon streetgive...
1,1,B'z,松本孝弘,稲葉浩志,あいかわらずなボクら,https://www.uta-net.com/song/7938/,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...,：1991-11-2,：BMCR-64B,今 好き こと し 我慢 し き あいつ ダメ キミ 勝手 決め 余計 お世話 どこ 行っ ...,今 好き 我慢 き あいつ ダメ キミ 勝手 決め 余計 お世話 どこ...
2,2,B'z,TAK MATSUMOTO,KOSHI INABA,愛と憎しみのハジマリ,https://www.uta-net.com/song/17979/,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...,：2003-09-1,：BMCV-800,誰 愛する 者 守ろ し てる 頃 憎しみ スキ うかがう 破綻 雰囲気 世界中 取り巻い ...,誰 愛する 者 守ろ 頃 憎しみ スキ うかがう 破綻 雰囲気 世界中 ...
3,3,B'z,松本孝弘,稲葉浩志,愛のバクダン,https://www.uta-net.com/song/25415/,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...,：2005-03-0,：BMCV-400,今日 一 日 不完全 世界 きみ 泣か 誰 泣く 人 こしらえ 悲しみ からくり できる の...,今日 一 日 不完全 世界 きみ 泣か 誰 泣く 人 こしらえ 悲しみ...
4,4,B'z,TAK MATSUMOTO,KOSHI INABA,愛のprisoner,https://www.uta-net.com/song/26049/,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど...",：2000-12-0,：BMCR-704,"I hate you , hate you こらえ きれ 夜 闇 吠える 涙 流さ あなた ...",hate hate こらえ きれ 夜 闇 吠える 涙 流さ あなた ...


# 不要語の削除

## 英語の不要語

In [57]:
from sklearn.feature_extraction import stop_words
stop_words_sklearn = stop_words.ENGLISH_STOP_WORDS

import nltk
nltk.download('stopwords')
stop_words_nltk = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/masato.ninohira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [58]:
len(stop_words_sklearn), len(stop_words_nltk)

(318, 179)

In [59]:
# 連結し重複ある単語をsetで削除
eng_stop_words = list(set(stop_words_sklearn.union(stop_words_nltk)))
len(eng_stop_words)

378

## 日本語の不要語

In [60]:
add_words = [
    ",", 'それ', 'てる', 'よう', 'こと', 'の', 'し', 'い', 'ん', 'さ', 'て', 'せ', 'れ'
]
len(add_words)

13

## 全不要語の削除

In [46]:
stop_words_all = eng_stop_words + add_words
len(stop_words_all)

391

In [63]:
def del_stop_words(text):
    #!マークなども消したいので正規表現で空文字に置換
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    sample = regex.sub('', text)

    #小文字にして、stop wordsに入っていなければリストに追加
    words = [w.lower()  for w in sample.split(' ') \
                 if not w.lower()  in stop_words_all]
    words = ' '.join(words)
    return words

In [64]:
df['remove_stop_words'] = list(df['preprocessed_by_mecab'].apply(lambda x: del_stop_words(x)))
df.head()

,Unnamed: 0,Artist,Composer,Lyricist,SongName,URL,Lyric,Sales_Date,CD_Number,preprocessed_by_mecab,remove_stop_words
0,0,B'z,Takahiro Matsumoto,Kohshi Inaba,I WANNA DANCE Wicked Beat Style,https://www.uta-net.com/song/54892/,Crush! Crush! Taxi on a busy neon streetGive m...,：1990-06-2,：BMCR-900,Crush ! Crush ! Taxi on a busy neon streetGive...,crush crush taxi busy neon streetgive baby ...
1,1,B'z,松本孝弘,稲葉浩志,あいかわらずなボクら,https://www.uta-net.com/song/7938/,今まで好きなこともしたし たまに我慢もしてきた「あいつはダメだ」 なんてキミ勝手に決めないで...,：1991-11-2,：BMCR-64B,今 好き こと し 我慢 し き あいつ ダメ キミ 勝手 決め 余計 お世話 どこ 行っ ...,今 好き 我慢 き あいつ ダメ キミ 勝手 決め 余計 お世話 どこ 行っ いい 道 いく...
2,2,B'z,TAK MATSUMOTO,KOSHI INABA,愛と憎しみのハジマリ,https://www.uta-net.com/song/17979/,誰もが愛する者 守ろうとしてる頃 憎しみがスキをうかがう大破綻の雰囲気が世界中を取り巻いて朝...,：2003-09-1,：BMCV-800,誰 愛する 者 守ろ し てる 頃 憎しみ スキ うかがう 破綻 雰囲気 世界中 取り巻い ...,誰 愛する 者 守ろ 頃 憎しみ スキ うかがう 破綻 雰囲気 世界中 取り巻い 朝夕 空 ...
3,3,B'z,松本孝弘,稲葉浩志,愛のバクダン,https://www.uta-net.com/song/25415/,今日も一日 不完全な世界きみが泣かなくても 誰か泣く人がこしらえた 悲しみのからくりなんとか...,：2005-03-0,：BMCV-400,今日 一 日 不完全 世界 きみ 泣か 誰 泣く 人 こしらえ 悲しみ からくり できる の...,今日 一 日 不完全 世界 きみ 泣か 誰 泣く 人 こしらえ 悲しみ からくり できる s...
4,4,B'z,TAK MATSUMOTO,KOSHI INABA,愛のprisoner,https://www.uta-net.com/song/26049/,"I hate you, hate you こらえきれずに夜の闇に吠える 涙流さないあなたのど...",：2000-12-0,：BMCR-704,"I hate you , hate you こらえ きれ 夜 闇 吠える 涙 流さ あなた ...",hate hate こらえ きれ 夜 闇 吠える 涙 流さ あなた どこ 念入り 探し 僕...


In [65]:
df.to_csv('preprocessed_lyrics.csv')